# MARVIC models with EFI standards

EFI standards are concerned with two main files:

1. The model output file which contains the actual predicted values from the model. This file will be in **csv** format. There is also a possibility to have the output file in netCDF format, but tabular data has certain advantages.

2. The dataset metadata file contains all the accompanying metadata information that will help interpret and use the model outputs (resolution, dimensions, variable names, units etc. but also uncertainty propagation, data assimilation, model complexity etc.). This file will be in **json** format.


## Start with installing libraries

Install and import Python equivalents of the R packages, including pandas, numpy, xarray for NetCDF handling (e.g. if your raw model outputs are in NetCDF), and pyarrow for Parquet files.


In [49]:
# Python standard library
import datetime
import tempfile
import xml.etree.ElementTree as ET

# External dependencies; all available on PyPI
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import dicttoxml
import xarray as xr
import json
import xmltodict

# optional
import pyarrow as pa
import pyarrow.dataset as ds


# Producing the model output file

## Read your model outputs

Read in your model outputs in your raw format. The example below assumes your outputs are in netcdf format:


In [ ]:
# Read Model Outputs from NetCDF

# Define the path to the NetCDF file
raw_output_file = ".../my_raw_model_output.nc"

# Open the NetCDF file using xarray
ds = xr.open_dataset(raw_output_file)

# Extract variables from the dataset
out_nee = ds["NEE"].values  # Net Ecosystem Exchange
out_gpp = ds["GPP"].values  # Gross Primary Productivity
out_ar = ds["AutoResp"].values  # Autotrophic Respiration

# Calculate Net Primary Productivity (NPP)
out_npp = out_gpp - out_ar

# Close the dataset
ds.close()

# Reformat Model Outputs to EFI Standards

Convert outputs to a flat file format (CSV):


In [ ]:
# Reformat Model Outputs to EFI Standards

# reference_datetime and pubtime can be the same for MARVIC
# just putting today's date
pub_datetime = reference_datetime = pd.Timestamp.today()

# Define the actual time stamps for the predictions
n_time = out_gpp.shape[1]
start_date = pd.Timestamp("2018-01-01")  # Replace with your simulation start date
datetimes = pd.date_range(start=start_date, periods=n_time, freq="D")

# Define ensemble parameters
n_ensembles = out_gpp.shape[0]
ensembles = np.arange(1, n_ensembles + 1)

obs_dim = 1  ## 1 = latent state
## 2 = latent state + observation error

variable_names = [
    "net_ecosystem_exchange",
    "gross_primary_productivity",
    "net_primary_productivity",
]

# Prepare storage for reformatted data
output_storage = np.empty((n_ensembles, n_time, len(variable_names)))

output_storage[..., 0] = np.squeeze(out_nee)
output_storage[..., 1] = np.squeeze(out_gpp)
output_storage[..., 2] = np.squeeze(out_npp)

# Define flags for data assimilation and forecast
data_assimilation = np.zeros(n_time, dtype=int)  # 0 indicates 'free-run'
forecast = np.zeros(n_time, dtype=int)  # 0 = hindcast, 1 = forecast

# Wrangle data from array into a long format DataFrame
df_combined = (
    pd.DataFrame(
        output_storage.reshape(-1, len(variable_names)), columns=variable_names
    )
    .assign(
        datetime=np.repeat(datetimes, n_ensembles),
        parameter=np.tile(ensembles, n_time),
        obs_flag=obs_dim,
    )
    .melt(
        id_vars=["datetime", "parameter", "obs_flag"],
        var_name="variable",
        value_name="prediction",
    )
)

# Add data assimilation and forecast flags
df_combined = df_combined.merge(
    pd.DataFrame(
        {
            "datetime": datetimes,
            "data_assimilation": data_assimilation,
            "forecast": forecast,
        }
    ),
    on="datetime",
)

# Display the first few rows of the reformatted DataFrame
print(df_combined.head())

    datetime  parameter  obs_flag                variable    prediction  \
0 2018-01-01          1         1  net_ecosystem_exchange  8.429057e-09   
1 2018-01-01          2         1  net_ecosystem_exchange  8.850567e-09   
2 2018-01-01          3         1  net_ecosystem_exchange  9.294480e-09   
3 2018-01-01          4         1  net_ecosystem_exchange  9.276512e-09   
4 2018-01-01          5         1  net_ecosystem_exchange  9.312278e-09   

   data_assimilation  forecast  
0                  0         0  
1                  0         0  
2                  0         0  
3                  0         0  
4                  0         0  


In [16]:
print(df_combined.tail())

          datetime  parameter  obs_flag                  variable  \
1682683 2023-12-31        252         1  net_primary_productivity   
1682684 2023-12-31        253         1  net_primary_productivity   
1682685 2023-12-31        254         1  net_primary_productivity   
1682686 2023-12-31        255         1  net_primary_productivity   
1682687 2023-12-31        256         1  net_primary_productivity   

           prediction  data_assimilation  forecast  
1682683 -1.631362e-09                  0         0  
1682684 -1.623129e-09                  0         0  
1682685 -1.659175e-09                  0         0  
1682686 -1.684502e-09                  0         0  
1682687 -1.538097e-09                  0         0  


## Write EFI csv

Export the formatted DataFrame to CSV using pandas and to Parquet format using pyarrow/pandas with appropriate compression.


In [ ]:
# Define the path for the CSV file
# YOUR PATH HERE
# we can discuss file naming
csv_filename = ".../FI-Qvd_SPY-C_2018-2023_EFIstandard.csv"

# Write the DataFrame to a CSV file
df_combined.to_csv(csv_filename, index=False)

# OPTIONAL
### convert to parquet format
# this is preliminary at the moment
# included here more as a discussion point
# FMI can convert your csv into parquet
# we are currently discussing the S3 folder hierarchy
# use cases to be discussed

# Define the path for the Parquet file
# YOUR PATH HERE
# parquet_path = ".../parquet_output"

# Write the DataFrame as a partitioned dataset
# ds.write_dataset(pa.Table.from_pandas(df_combined), parquet_path, format="parquet")

# Producing the standard Metadata file

EFI metadata convention builds on the Ecological Metadata Language (EML) metadata standard. EML has a long development history, is interconvertible with many other standards, and has built-in extensibility.

EFI dataset metadata convention makes some core components of the base EML standard required or recommended. Many optional elements also exist as part of the EML schema (https://eml.ecoinformatics.org/schema/). The appendices of the EFI standards publication (https://doi.org/10.1002/ecs2.4686) includes the descriptions of these tags in more detail. We made these pdfs available on MARVIC share point also.

## Prepare dataset metadata

In R, there is the EML package but in python there is no package/function, so we have to generate the structures by hand:


In [26]:
# NOTE: List of dicts
attrList = [
    {
        "attributeName": "datetime",
        "attributeDefinition": "[dimension]{datetime}",
        "storageType": "date",
        "measurementScale": {
            "dateTime": {"formatString": "YYYY-MM-DD", "dateTimeDomain": []}
        },
    },
    {
        "attributeName": "parameter",
        "attributeDefinition": "[dimension]{index of ensemble member}",
        "storageType": "float",
        "measurementScale": {
            "ratio": {
                "unit": {"standardUnit": "dimensionless"},
                "numericDomain": {"numberType": "integer"},
            }
        },
    },
    {
        "attributeName": "obs_flag",
        "attributeDefinition": "[dimension]{observation error}",
        "storageType": "float",
        "measurementScale": {
            "ratio": {
                "unit": {"standardUnit": "dimensionless"},
                "numericDomain": {"numberType": "integer"},
            }
        },
    },
    {
        "attributeName": "net_ecosystem_exchange",
        "attributeDefinition": "[variable]{Net Ecosystem Exchange}",
        "storageType": "float",
        "measurementScale": {
            "ratio": {
                "unit": {"standardUnit": "kg C m-2 s-1"},
                "numericDomain": {"numberType": "real"},
            }
        },
    },
    {
        "attributeName": "gross_primary_productivity",
        "attributeDefinition": "[variable]{Gross Primary Productivity}",
        "storageType": "float",
        "measurementScale": {
            "ratio": {
                "unit": {"standardUnit": "kg C m-2 s-1"},
                "numericDomain": {"numberType": "real"},
            }
        },
    },
    {
        "attributeName": "data_assimilation",
        "attributeDefinition": "[flag]{whether time step assimilated data}",
        "storageType": "float",
        "measurementScale": {
            "ratio": {
                "unit": {"standardUnit": "dimensionless"},
                "numericDomain": {"numberType": "integer"},
            }
        },
    },
    {
        "attributeName": "forecast",
        "attributeDefinition": "[flag]{whether time step forecast or hindcast}",
        "storageType": "float",
        "measurementScale": {
            "ratio": {
                "unit": {"standardUnit": "dimensionless"},
                "numericDomain": {"numberType": "integer"},
            }
        },
    },
]

In [ ]:
physical = {
    "objectName": ".../FI-Qvd_SPY-C_2018-2023_EFIstandard.csv",
    "size": {"unit": "bytes", "size": "110780"},
    "authentication": {
        "method": "MD5",
        "authentication": "4dbe687fa1f5fc0ff789096076eebd78",
    },
    "dataFormat": {
        "textFormat": {
            "recordDelimiter": "\n",
            "attributeOrientation": "column",
            "simpleDelimited": {"fieldDelimiter": ","},
        }
    },
}

dataTable = {
    "entityName": "MARVIC T3.2? outputs",  ## this is a standard name to allow us to distinguish this entity from
    "entityDescription": "Agro-ecosystem carbon budget predictions",
    "physical": physical,
    "attributeList": attrList,
}

In [38]:
# who to contact about this output
creator_info = {
    "individualName": {"givenName": "YourName", "surName": "YourSurname"},
    "electronicMailAddress": "YourEmail",
    "id": "https://orcid.org/YourOrcid",
}

# # can also set taxonomic, temporal, and geographic coverage of the outputs
# UPDATE ACCORDINGLY
coverage = {
    "geographicCoverage": {
        "geographicDescription": "Qvidja, Finland",
        "boundingCoordinates": {
            "westBoundingCoordinate": 22.39017,
            "eastBoundingCoordinate": 22.3932251,
            "northBoundingCoordinate": 60.29531,
            "southBoundingCoordinate": 60.29331,
        },
    },
    "temporalCoverage": {
        "rangeOfDates": {
            "beginDate": {"calendarDate": str(datetimes[0])},
            "endDate": {"calendarDate": str(datetimes[-1])},
        }
    },
    "taxonomicCoverage": {
        "taxonomicClassification": {
            "taxonRankName": "Genus",
            "taxonRankValue": "Triticum",
            "taxonomicClassification": {
                "taxonRankName": "Species",
                "taxonRankValue": "aestivum",
            },
        },
        "taxonomicClassification": {
            "taxonRankName": "Genus",
            "taxonRankValue": "Phleum",
            "taxonomicClassification": {
                "taxonRankName": "Species",
                "taxonRankValue": "pratense",
            },
        },
    },
}

# # Set key words.  We will need to develop a MARVIC controlled vocabulary
keywordSet = [
    {
        "keywordThesaurus": "MARVIC controlled vocabulary",
        "keyword": ["hindcast", "fluxes", "timeseries"],
    }
]

Combine the above bits to document the output dataset as a whole:


In [41]:
dataset = {
    "title": "MARVIC T3.2 model outputs",
    "creator": creator_info,
    "pubDate": reference_datetime,
    "intellectualRights": "https://creativecommons.org/licenses/by/4.0/",
    "abstract": "An illustration of how we might use EML metadata to describe a MARVIC output",
    "dataTable": dataTable,
    "keywordSet": keywordSet,
    "coverage": coverage,
}

## Prepare additional metadata

EFI standards has additional metadata tags to store forecast specific information for basic elements and model structure & uncertainty:

- `target_id`: this is a unique identifier (e.g., URL or DOI) that links to data or metadata about what the forecast/prediction is being scored against

- `model_version`: This identifier should update when the model is updated or when the underlying modeling **workflow** is updated. EFI recommends issuing DOIs for different model/workflow versions, and thus, this is a natural choice for a model_version.

- `iteration_id`: represents a unique ID for each run. Examples might be a start time or database ID.


In [ ]:
# Define additional metadata for EFI standards
## Global attributes
target_id = "YOUR_TARGET_ID"
model_name = "YOUR_MODEL_NAME"
model_version = "YOUR_MODEL_VERSION"
iteration_id = "20180101T000000"  # UPDATE ACCORDINGLY

my_horizon = f"{n_time} days"  # UPDATE ACCORDINGLY

additionalMetadata = {
    "metadata": {
        "forecast": {
            ## Basic elements
            "timestep": "1 day",  ## should be udunits parsable; change if not days
            "horizon": my_horizon,
            "reference_datetime": reference_datetime,
            "iteration_id": iteration_id,
            "target_id": target_id,
            "metadata_standard_version": "1.0",
            "model_description": {
                "model_id": model_version,
                "name": model_name,
                "type": "process-based",
                "repository": "https://github.com/YOUR_REPO",
            },
            ## MODEL STRUCTURE & UNCERTAINTY CLASSES
            "initial_conditions": {"present": False},
            "drivers": {"present": False},
            "parameters": {
                "present": True,
                "date_driven": True,
                "complexity": 4,  ## number of parameters being varied, UPDATE ACCORDINGLY
                "propagation": {"type": "ensemble", "size": 256},  # UPDATE ACCORDINGLY
            },
            "random_effects": {"present": False},
            "process_error": {"present": False},
            "obs_error": {"present": False},
        }  # forecast
    }  # metadata
}


Write metadata file: In python, there are no nice utilities, so we have to create the XML by hand.
This is significantly simplified by the `dicttoxml` package, though we still have to do a bit of extra work.


In [ ]:
dataset_xml = ET.fromstring(
    dicttoxml.dicttoxml(dataset, custom_root="dataset", attr_type=False)
)

metadata_xml = ET.fromstring(
    dicttoxml.dicttoxml(
        additionalMetadata, custom_root="additionalMetadata", attr_type=False
    )
)

my_eml = ET.Element(
    "eml:eml",
    {
        "xmlns:eml": "https://eml.ecoinformatics.org/eml-2.2.0",
        "xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
        "xmlns:stmml": "http://www.xml-cml.org/schema/stmml-1.2",
        "system": "datetime",
        "packageId": iteration_id,
        "xsi:schemaLocation": "https://eml.ecoinformatics.org/eml-2.2.0 https://eml.ecoinformatics.org/eml-2.2.0/eml.xsd",
    },
)

my_eml.append(dataset_xml)
my_eml.append(metadata_xml)

In [ ]:
# Write metadata to XML and JSON files
eml_filename = ".../my_model.xml"
with open(eml_filename, "w") as f:
    f.write(ET.tostring(my_eml, encoding="unicode"))

# Convert XML to JSON
xml_string = ET.tostring(my_eml, encoding="unicode")
xml_dict = xmltodict.parse(xml_string)

# Write to JSON file
json_filename = ".../my_model.json"
with open(json_filename, "w") as f:
    json.dump(xml_dict, f, indent=2)